In [ ]:
from colabfit.tools.database import MongoDatabase, load_data
from colabfit.tools.property_settings import PropertySettings

client = MongoDatabase('colabfit_database', nprocs=1)

# Number of configurations

In [ ]:
client.configurations.count_documents({})

# Number of properties

In [ ]:
client.properties.count_documents({})

# Number of datasets

In [ ]:
client.datasets.count_documents({})

## Names of datasets

In [ ]:
sorted(list(client.datasets.find({}, {'name'})), key=lambda x: x['name'].lower())

# Number of configuration sets

In [ ]:
client.configuration_sets.count_documents({})

## All configuration sets, and their linked datasets

In [ ]:
cursor = client.configuration_sets.aggregate([
    {'$project': {'relationships.datasets': 1}},
    {'$unwind': '$relationships.datasets'},
    {'$project': {'ds_id': '$relationships.datasets'}},
    {'$lookup': {
        'from': 'datasets',
        'localField': 'ds_id',
        'foreignField': '_id',
        'as': 'linked_ds'
    }},
    {'$project': {'ds_name': '$linked_ds.name'}}
])

sorted(list(cursor), key=lambda x: x['ds_name'][0].lower())

## Configuration sets that are tied to more than one dataset

In [ ]:
client.configuration_sets.count_documents({'relationships.datasets.1': {'$exists': True}})

In [ ]:
cursor = client.configuration_sets.aggregate([
    {'$match': {'relationships.datasets.1': {'$exists': True}}},
    {'$project': {'relationships.datasets': 1}},
    {'$unwind': '$relationships.datasets'},
    {'$project': {'ds_id': '$relationships.datasets'}},
    {'$lookup': {
        'from': 'datasets',
        'localField': 'ds_id',
        'foreignField': '_id',
        'as': 'linked_ds'
    }},
    {'$project': {'ds_name': '$linked_ds.name'}}
])

list(cursor)

# Total number of atoms

In [ ]:
next(client.configurations.aggregate([
    {'$group': {'_id': None, 'sum': {'$sum': '$nsites'}}}
]))

# Total number of configuration labels

In [ ]:
cursor = client.configurations.aggregate([
#     { "$match": { "_id": { "$in": [1, 2] } } },
    {"$group": {"_id": None, "labels": { "$push": "$labels" }}},
    {
        "$project": {
            "labels": {
                "$reduce": {
                    "input": "$labels",
                    "initialValue": [],
                    "in": { "$setUnion": ["$$value", "$$this"] }
                }
            }
        }
    }
])

labels = next(cursor)['labels']
len(labels)

## Counts for each label

In [ ]:
cursor = client.configurations.aggregate([
    {'$unwind': '$labels'},
    {'$group': {'_id': '$labels', 'count': {'$sum': 1}}}
])

sorted(cursor, key=lambda x: x['count'], reverse=True)